#### *This notebook is meant to be run on the RSP*

This notebook loads photometry for DP1 objects that have SNR > 5 in the $i$ band and save them in a fits table.  
It also loads the DP1 photo-z estimates, and saves them, without photometry, in a dataframe.

In [ ]:
from lsst.rsp import get_tap_service

In [ ]:
# Get the TAP service
service = get_tap_service("tap")

In [ ]:
query = """
SELECT
    objectID,
    coord_ra,
    coord_dec,
    refBand,
    ebv,
    u_psfFlux,
    u_psfFluxErr,
    g_psfFlux,
    g_psfFluxErr,
    r_psfFlux,
    r_psfFluxErr,
    i_psfFlux,
    i_psfFluxErr,
    z_psfFlux,
    z_psfFluxErr,
    y_psfFlux,
    y_psfFluxErr,
    u_cModelFlux,
    u_cModelFluxErr,
    g_cModelFlux,
    g_cModelFluxErr,
    r_cModelFlux,
    r_cModelFluxErr,
    i_cModelFlux,
    i_cModelFluxErr,
    z_cModelFlux,
    z_cModelFluxErr,
    y_cModelFlux,
    y_cModelFluxErr,
    scisql_nanojanskyToAbMag(u_cModelFlux) AS u_cModelMag,
    scisql_nanojanskyToAbMagSigma(u_cModelFlux, u_cModelFluxErr) AS u_cModelMagErr,
    scisql_nanojanskyToAbMag(g_cModelFlux) AS g_cModelMag,
    scisql_nanojanskyToAbMagSigma(g_cModelFlux, g_cModelFluxErr) AS g_cModelMagErr,
    scisql_nanojanskyToAbMag(r_cModelFlux) AS r_cModelMag,
    scisql_nanojanskyToAbMagSigma(r_cModelFlux, r_cModelFluxErr) AS r_cModelMagErr,
    scisql_nanojanskyToAbMag(i_cModelFlux) AS i_cModelMag,
    scisql_nanojanskyToAbMagSigma(i_cModelFlux, i_cModelFluxErr) AS i_cModelMagErr,
    scisql_nanojanskyToAbMag(z_cModelFlux) AS z_cModelMag,
    scisql_nanojanskyToAbMagSigma(z_cModelFlux, z_cModelFluxErr) AS z_cModelMagErr,
    scisql_nanojanskyToAbMag(y_cModelFlux) AS y_cModelMag,
    scisql_nanojanskyToAbMagSigma(y_cModelFlux, y_cModelFluxErr) AS y_cModelMagErr,
    scisql_nanojanskyToAbMag(u_gaap1p0Flux) AS u_gaap1p0Mag,
    scisql_nanojanskyToAbMagSigma(u_gaap1p0Flux, u_gaap1p0FluxErr) AS u_gaap1p0MagErr,
    scisql_nanojanskyToAbMag(g_gaap1p0Flux) AS g_gaap1p0Mag,
    scisql_nanojanskyToAbMagSigma(g_gaap1p0Flux, g_gaap1p0FluxErr) AS g_gaap1p0MagErr,
    scisql_nanojanskyToAbMag(r_gaap1p0Flux) AS r_gaap1p0Mag,
    scisql_nanojanskyToAbMagSigma(r_gaap1p0Flux, r_gaap1p0FluxErr) AS r_gaap1p0MagErr,
    scisql_nanojanskyToAbMag(i_gaap1p0Flux) AS i_gaap1p0Mag,
    scisql_nanojanskyToAbMagSigma(i_gaap1p0Flux, i_gaap1p0FluxErr) AS i_gaap1p0MagErr,
    scisql_nanojanskyToAbMag(z_gaap1p0Flux) AS z_gaap1p0Mag,
    scisql_nanojanskyToAbMagSigma(z_gaap1p0Flux, z_gaap1p0FluxErr) AS z_gaap1p0MagErr,
    scisql_nanojanskyToAbMag(y_gaap1p0Flux) AS y_gaap1p0Mag,
    scisql_nanojanskyToAbMagSigma(y_gaap1p0Flux, y_gaap1p0FluxErr) AS y_gaap1p0MagErr
FROM dp1.Object
WHERE
    i_cModelFlux / i_cModelFluxErr > 5 AND
    u_cModel_flag = 0 AND
    g_cModel_flag = 0 AND
    r_cModel_flag = 0 AND
    i_cModel_flag = 0 AND
    z_cModel_flag = 0 AND
    y_cModel_flag = 0 AND
    u_gaapFlux_flag = 0 AND
    g_gaapFlux_flag = 0 AND
    r_gaapFlux_flag = 0 AND
    i_gaapFlux_flag = 0 AND
    z_gaapFlux_flag = 0 AND
    y_gaapFlux_flag = 0 AND
    CONTAINS(POINT('ICRS', coord_ra, coord_dec), CIRCLE('ICRS', 53.1, -28.1, 0.75)) = 1
"""
# Last query selects only ECDFS

In [ ]:
job = service.submit_job(query)
job.run()
job.wait(phases=["COMPLETED", "ERROR"])
print("Job phase is", job.phase)
job.raise_if_error()

In [ ]:
results = job.fetch_result()
cat = results.to_table()
cat

In [ ]:
cat.write("../data/dp1_ecdfs_raw.fits")

### *Now* the DP1 photo-z's

In [ ]:
from upath import UPath
import lsdb

In [ ]:
# Load the catalog as saved in LSDB
base_path = UPath("/rubin/lsdb_data")
dp1_pz_catalog = lsdb.open_catalog(base_path / "object_photoz")
dp1_pz_catalog = dp1_pz_catalog.compute().to_pandas()

# Now cut out the mag columns
dp1_pz_catalog = dp1_pz_catalog[[col for col in obj.columns if "Mag" not in col]]

# Save
dp1_pz_catalog.to_parquet("../data/photozs.parquet")

dp1_pz_catalog